In [124]:
# -*- coding: utf-8 -*-
import os
import sys
import time
import pandas as pd
import numpy as np
import torch
from torch.optim import Adagrad
from sklearn.metrics import log_loss, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

sys.path.append("/Users/wzq/Desktop/game")
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *
from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

from utils import save_zip_file

In [133]:
# data = pd.read_csv('./data/train_processed_product_name.csv')
# test_data = pd.read_csv('./data/test_processed_product_name.csv')
data = pd.read_csv('./data/train_processed_bls_cls.csv')
test_data = pd.read_csv('./data/test_processed_bls_cls.csv')
print(data.shape)
print(test_data.shape)
data = pd.concat([data, test_data], axis=0)
print(data.shape)
data

(745889, 78)
(223553, 75)
(969442, 78)


,Unnamed: 0,user_id,product_id,product_name,rating,votes,helpful_votes,ID,user_rating_mean,user_rating_count,...,product_name_embeddings_40,product_name_embeddings_41,product_name_embeddings_42,product_name_embeddings_43,product_name_embeddings_44,product_name_embeddings_45,product_name_embeddings_46,product_name_embeddings_47,product_name_embeddings_48,product_name_embeddings_49
0,0,1813,154533,Beautiful Thing,5.0,10.0,8.0,0,4.001621,617,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
1,1,1916,154533,Beautiful Thing,4.0,2.0,2.0,681987,4.241384,121089,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
2,2,506,154533,Beautiful Thing,5.0,6.0,6.0,288065,4.763975,483,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
3,3,923,154533,Beautiful Thing,5.0,9.0,8.0,365036,4.261745,149,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
4,4,1560,154533,Beautiful Thing,4.0,6.0,5.0,237732,3.986111,792,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223548,223548,1916,214704,"Godzilla, King of the Monsters",NaN,NaN,NaN,223548,4.241384,121089,...,-0.232975,0.261822,-0.117066,-0.085076,-0.090386,0.542517,-0.008391,-0.779783,-0.190348,-0.343293
223549,223549,1502,169489,Houses of the Holy,NaN,NaN,NaN,223549,4.052681,31131,...,-0.204035,-0.123490,-0.158017,0.257991,-0.072135,0.440195,-0.513949,-0.577805,-0.299442,-0.928587
223550,223550,874,150908,Frankenstein and the Monster from Hell,NaN,NaN,NaN,223550,4.393426,4898,...,-0.234717,0.238423,-0.246091,-0.245436,0.008666,0.332809,0.197266,-0.858108,0.037638,-0.281256
223551,223551,922,198737,X-Men,NaN,NaN,NaN,223551,4.561964,1283,...,0.177770,-0.888020,-0.477400,-0.891320,-0.850960,1.257500,0.745990,-0.560890,-0.162060,-0.417890


In [134]:
sparse_features = ['user_id', 'product_id']

dense_features = ['user_rating_mean', 'user_votes_mean', 'user_helpful_votes_mean'] \
                + ['product_rating_mean', 'product_votes_mean','product_helpful_votes_mean'] \
                + ['user_votes_helpful_rate', 'user_helpful_rating'] \
                + ['product_votes_helpful_rate', 'product_helpful_rating'] \
                + ['product_name_embeddings_' + str(i) for i in range(50)]
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['rating']
data

,Unnamed: 0,user_id,product_id,product_name,rating,votes,helpful_votes,ID,user_rating_mean,user_rating_count,...,product_name_embeddings_40,product_name_embeddings_41,product_name_embeddings_42,product_name_embeddings_43,product_name_embeddings_44,product_name_embeddings_45,product_name_embeddings_46,product_name_embeddings_47,product_name_embeddings_48,product_name_embeddings_49
0,0,1813,154533,Beautiful Thing,5.0,10.0,8.0,0,4.001621,617,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
1,1,1916,154533,Beautiful Thing,4.0,2.0,2.0,681987,4.241384,121089,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
2,2,506,154533,Beautiful Thing,5.0,6.0,6.0,288065,4.763975,483,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
3,3,923,154533,Beautiful Thing,5.0,9.0,8.0,365036,4.261745,149,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
4,4,1560,154533,Beautiful Thing,4.0,6.0,5.0,237732,3.986111,792,...,-0.378545,0.126550,0.087245,0.062750,0.059166,0.266091,0.013202,-0.767775,0.244955,0.535740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223548,223548,1916,214704,"Godzilla, King of the Monsters",NaN,NaN,NaN,223548,4.241384,121089,...,-0.232975,0.261822,-0.117066,-0.085076,-0.090386,0.542517,-0.008391,-0.779783,-0.190348,-0.343293
223549,223549,1502,169489,Houses of the Holy,NaN,NaN,NaN,223549,4.052681,31131,...,-0.204035,-0.123490,-0.158017,0.257991,-0.072135,0.440195,-0.513949,-0.577805,-0.299442,-0.928587
223550,223550,874,150908,Frankenstein and the Monster from Hell,NaN,NaN,NaN,223550,4.393426,4898,...,-0.234717,0.238423,-0.246091,-0.245436,0.008666,0.332809,0.197266,-0.858108,0.037638,-0.281256
223551,223551,922,198737,X-Men,NaN,NaN,NaN,223551,4.561964,1283,...,0.177770,-0.888020,-0.477400,-0.891320,-0.850960,1.257500,0.745990,-0.560890,-0.162060,-0.417890


In [135]:
# 对稀疏的类别特征进行简单的处理，处理为连续整数类别
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [136]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4) for feat in sparse_features] \
                       + [DenseFeat(feat, 1, )for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
feature_names

['user_id',
 'product_id',
 'user_rating_mean',
 'user_votes_mean',
 'user_helpful_votes_mean',
 'product_rating_mean',
 'product_votes_mean',
 'product_helpful_votes_mean',
 'user_votes_helpful_rate',
 'user_helpful_rating',
 'product_votes_helpful_rate',
 'product_helpful_rating',
 'product_name_embeddings_0',
 'product_name_embeddings_1',
 'product_name_embeddings_2',
 'product_name_embeddings_3',
 'product_name_embeddings_4',
 'product_name_embeddings_5',
 'product_name_embeddings_6',
 'product_name_embeddings_7',
 'product_name_embeddings_8',
 'product_name_embeddings_9',
 'product_name_embeddings_10',
 'product_name_embeddings_11',
 'product_name_embeddings_12',
 'product_name_embeddings_13',
 'product_name_embeddings_14',
 'product_name_embeddings_15',
 'product_name_embeddings_16',
 'product_name_embeddings_17',
 'product_name_embeddings_18',
 'product_name_embeddings_19',
 'product_name_embeddings_20',
 'product_name_embeddings_21',
 'product_name_embeddings_22',
 'product_nam

In [137]:
# 3.generate input data for model
test_predict = data.iloc[-test_data.shape[0]:, :] # (223553, 28)
train, test = train_test_split(data.iloc[:-test_data.shape[0], :], test_size=0.2, random_state=2020)  # 删除测试数据集 (745889, 28)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
test_predict_input = {name: test_predict[name] for name in feature_names}

In [139]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = False
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#             dnn_hidden_units=(256,128), l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, 
#             dnn_dropout=0.5, dnn_activation='relu', dnn_use_bn=True, task='regression')
model = DCN(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, cross_num=2,
            dnn_hidden_units=(128,128), l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_cross=0.00001, l2_reg_dnn=0, 
            dnn_dropout=0.5, dnn_activation='relu', dnn_use_bn=False, task='regression')
model.compile(Adagrad(model.parameters(),0.001), loss="mse", metrics=["mse"])

es = EarlyStopping(monitor='val_mse', min_delta=0, verbose=1, patience=5, mode='min')
mdckpt = ModelCheckpoint(filepath='best_models/DCN_model.ckpt', monitor='val_mse', verbose=1, save_best_only=True, mode='min')
history = model.fit(train_model_input, train[target].values, batch_size=5120, epochs=2000, verbose=2, validation_split=0.2, callbacks=[es,mdckpt])
model = torch.load('best_models/DCN_model.ckpt')

pred_ans = model.predict(test_model_input, 5120)
print("")
print("test mse", round(mean_squared_error(test[target].values, pred_ans), 4))
print("test rmse", round(np.sqrt(mean_squared_error(test[target].values, pred_ans)), 4))

print(f"=========infer========={time.strftime('%Y%m%d%H%M', time.localtime())}")
outputs = model.predict(test_predict_input, 5120)
test_data['rating'] =  outputs
submission = test_data[['ID', 'rating']]
save_zip_file(submission, )
print(f"=========infer done========={time.strftime('%Y%m%d%H%M', time.localtime())}")
submission

cpu
Train on 477368 samples, validate on 119343 samples, 94 steps per epoch
Epoch 1/2000
5s - loss:  7.6737 - mse:  7.6262 - val_mse:  1.6833
Epoch 00001: val_mse improved from inf to 1.68330, saving model to best_models/DCN_model.ckpt
Epoch 2/2000
4s - loss:  1.1658 - mse:  1.1654 - val_mse:  1.0422
Epoch 00002: val_mse improved from 1.68330 to 1.04219, saving model to best_models/DCN_model.ckpt
Epoch 3/2000
4s - loss:  1.0391 - mse:  1.0396 - val_mse:  1.0337
Epoch 00003: val_mse improved from 1.04219 to 1.03368, saving model to best_models/DCN_model.ckpt
Epoch 4/2000
4s - loss:  1.0316 - mse:  1.0330 - val_mse:  1.0279
Epoch 00004: val_mse improved from 1.03368 to 1.02787, saving model to best_models/DCN_model.ckpt
Epoch 5/2000
5s - loss:  1.0249 - mse:  1.0248 - val_mse:  1.0223
Epoch 00005: val_mse improved from 1.02787 to 1.02226, saving model to best_models/DCN_model.ckpt
Epoch 6/2000
4s - loss:  1.0183 - mse:  1.0182 - val_mse:  1.0168
Epoch 00006: val_mse improved from 1.02226

,ID,rating
0,0,4.144410
1,1,3.004165
2,2,4.343686
3,3,3.802331
4,4,4.034181
...,...,...
223548,223548,4.062070
223549,223549,4.375775
223550,223550,3.616317
223551,223551,4.928766
